# TA-Lib Tutorial

"TA-Lib is widely used by trading software developers requiring to perform technical analysis of financial market data."

In [5]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [6]:
# imports
import pandas as pd
import pandas_datareader.data as pdr
import datetime
import talib
from talib.abstract import *
from talib import MA_Type

# format price data
pd.options.display.float_format = '{:0.2f}'.format

Some global data

In [32]:
l = ['SPY', 'AAPL', 'PLUG', 'MSFT', 'PENN', 'EBAY', 'GFI', 'ATVI', 'AMAT', 'TSM', 'MRVL', 'SE', 'CRM', 'LLY', 'ABB', 'DHI', 'FAST', 'MAS', 'CDNS', 'TBIO', 'GIS', 'IMMU', 'HD', 'STM', 'NUAN', 'LAKE', 'DT', 'HOLX', 'DG', 'BLDP', 'ADBE', 'OSTK', 'AU', 'BMRN', 'KNX', 'BJ', 'ADI', 'ADSK', 'PAAS', 'AHPI', 'CLX', 'LRCX', 'DLR', 'MKC', 'TEAM', 'INSM', 'A', 'SWKS', 'CLGX', 'CCI', 'CTXS', 'CMC', 'LULU', 'CMI', 'LSCC', 'WGO', 'VRTX', 'PRTS', 'KLAC', 'FTNT', 'TREX', 'ENTG', 'TAL', 'MNTA', 'DXCM', 'HALO', 'NBIX', 'IIVI', 'AVEO', 'EBS', 'SGEN', 'AMKR', 'THO', 'TSCO', 'RMD', 'REGN', 'ALNY', 'QDEL', 'CHTR', 'OTEX', 'WERN', 'BKI', 'ADPT', 'BLK', 'SNPS', 'WSM', 'FORM', 'AMRC', 'ILMN', 'MCO', 'CGEN', 'MOH', 'NTES', 'ROK', 'PEGA', 'NVDA', 'NEM', 'LOW', 'GLUU', 'SAP']
symbol = l[0]
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime.now()

Fetch timeseries

In [33]:
ts = pdr.DataReader(symbol, 'yahoo', start, end)
ts.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-11-17,394.95,390.14,390.46,394.24,74496300.00,394.24
2022-11-18,397.81,393.04,397.74,396.03,92783500.00,396.03
2022-11-21,395.82,392.66,394.64,394.59,51243200.00,394.59
2022-11-22,400.07,395.15,396.63,399.90,60429000.00,399.90
2022-11-23,402.93,399.31,399.55,402.42,68161400.00,402.42


In [35]:
def _adj_column_names(ts):
    """
    ta-lib expects columns to be lower case; to be consistent,
    change date index
    """
    ts.columns = [col.lower().replace(' ','_') for col in ts.columns]
    ts.index.names = ['date']
    return ts

ts = _adj_column_names(ts)


Select timeseries between start and end.

### Get info about TA-Lib

In [10]:
print('There are {} TA-Lib functions!'.format(len(talib.get_functions())))

There are 158 TA-Lib functions!


Here is a complete listing of the functions by group:

In [11]:
for group, funcs in talib.get_function_groups().items():
    print(group)
    print('-----------------------------------------')
    for func in funcs:
        f = Function(func)
        print('{} - {}'.format(func, f.info['display_name']))
    print()

Cycle Indicators
-----------------------------------------
HT_DCPERIOD - Hilbert Transform - Dominant Cycle Period
HT_DCPHASE - Hilbert Transform - Dominant Cycle Phase
HT_PHASOR - Hilbert Transform - Phasor Components
HT_SINE - Hilbert Transform - SineWave
HT_TRENDMODE - Hilbert Transform - Trend vs Cycle Mode

Math Operators
-----------------------------------------
ADD - Vector Arithmetic Add
DIV - Vector Arithmetic Div
MAX - Highest value over a specified period
MAXINDEX - Index of highest value over a specified period
MIN - Lowest value over a specified period
MININDEX - Index of lowest value over a specified period
MINMAX - Lowest and highest values over a specified period
MINMAXINDEX - Indexes of lowest and highest values over a specified period
MULT - Vector Arithmetic Mult
SUB - Vector Arithmetic Substraction
SUM - Summation

Math Transform
-----------------------------------------
ACOS - Vector Trigonometric ACos
ASIN - Vector Trigonometric ASin
ATAN - Vector Trigonometric AT

### Get info about a specific TA-Lib function

There are 2 different API that are available with talib, namely Function API and Abstract API.  For the Function API, you pass in a price series.  For the Abstract API, you pass in a collection of named inputs: 'open', 'high', 'low', 'close', and 'volume'.  One or more of these may be used as defaults, but can be changed with the 'price' parameter.  

Print the function instance to get documentation.  We see that SMA has the parameter 'timeperiod' with default '30'.  The input_arrays can be a dataframe with columns named 'open', 'high', 'low', 'close', and 'volume'.

In [12]:
print(SMA)

SMA([input_arrays], [timeperiod=30])

Simple Moving Average (Overlap Studies)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 30
Outputs:
    real


More information is available through the 'info' property.  We observe here that the default price used is 'close'.  This can be changed by setting 'price' in the function call, e.g. price='open'.

In [13]:
print(SMA.info)

{'name': 'SMA', 'group': 'Overlap Studies', 'display_name': 'Simple Moving Average', 'function_flags': ['Output scale same as input'], 'input_names': OrderedDict([('price', 'close')]), 'parameters': OrderedDict([('timeperiod', 30)]), 'output_flags': OrderedDict([('real', ['Line'])]), 'output_names': ['real']}


If we just want to see the inputs, we can print the input_names property.

In [14]:
print(SMA.input_names)

OrderedDict([('price', 'close')])


If we just want to see the parameters, we can print the paramters property.

In [15]:
print(SMA.parameters)

OrderedDict([('timeperiod', 30)])


If we just want to see the outputs, we can print the output_names property.

In [16]:
print(SMA.output_names)

['real']


### Create a technical indicator using talib

Create technical indicator: SMA (using defaults: timeperiod=30, price='close')

In [17]:
sma = SMA(ts)
sma.tail()

date
2022-11-17   377.57
2022-11-18   378.68
2022-11-21   379.83
2022-11-22   381.24
2022-11-23   382.77
dtype: float64

Create technical indicator: SMA (using: timeperiod=200, price='close')

In [18]:
sma200 = SMA(ts, timeperiod=200)
sma200.tail()

date
2022-11-17   406.06
2022-11-18   405.81
2022-11-21   405.54
2022-11-22   405.30
2022-11-23   405.06
dtype: float64

Create technical indicator: SMA (using: timeperiod=200, price='open')

In [19]:
sma200 = SMA(ts, timeperiod=200, price='open')
sma200.tail()

date
2022-11-17   406.09
2022-11-18   405.83
2022-11-21   405.57
2022-11-22   405.30
2022-11-23   405.07
dtype: float64

### Add a technical indicator to a timeseries

In [20]:
ts['sma200'] = sma200
ts.tail()

,high,low,open,close,volume,adj_close,sma200
date,,,,,,,
2022-11-17,394.95,390.14,390.46,394.24,74496300.00,394.24,406.09
2022-11-18,397.81,393.04,397.74,396.03,92783500.00,396.03,405.83
2022-11-21,395.82,392.66,394.64,394.59,51243200.00,394.59,405.57
2022-11-22,400.07,395.15,396.63,399.90,60429000.00,399.90,405.30
2022-11-23,402.93,399.31,399.55,402.42,68161400.00,402.42,405.07


### Try another one

Commodity Channel Index

In [21]:
print(CCI)

CCI([input_arrays], [timeperiod=14])

Commodity Channel Index (Momentum Indicators)

Inputs:
    prices: ['high', 'low', 'close']
Parameters:
    timeperiod: 14
Outputs:
    real


In [22]:
print(CCI.input_names)

OrderedDict([('prices', ['high', 'low', 'close'])])


In [23]:
print(CCI.parameters)

OrderedDict([('timeperiod', 14)])


In [24]:
cci = CCI(ts)
ts['cci'] = cci
ts.tail()

,high,low,open,close,volume,adj_close,sma200,cci
date,,,,,,,,
2022-11-17,394.95,390.14,390.46,394.24,74496300.00,394.24,406.09,57.02
2022-11-18,397.81,393.04,397.74,396.03,92783500.00,396.03,405.83,67.28
2022-11-21,395.82,392.66,394.64,394.59,51243200.00,394.59,405.57,50.60
2022-11-22,400.07,395.15,396.63,399.90,60429000.00,399.90,405.30,71.92
2022-11-23,402.93,399.31,399.55,402.42,68161400.00,402.42,405.07,93.21


### Now for something a little more difficult

Bollinger Bands

In [25]:
print(BBANDS)

BBANDS([input_arrays], [timeperiod=5], [nbdevup=2], [nbdevdn=2], [matype=0])

Bollinger Bands (Overlap Studies)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 5
    nbdevup: 2
    nbdevdn: 2
    matype: 0 (Simple Moving Average)
Outputs:
    upperband
    middleband
    lowerband


In [26]:
print(BBANDS.input_names)

OrderedDict([('price', 'close')])


In [27]:
print(BBANDS.parameters)

OrderedDict([('timeperiod', 5), ('nbdevup', 2), ('nbdevdn', 2), ('matype', 0)])


Print the available moving average types

In [28]:
attributes = [attr for attr in dir(MA_Type) 
              if not attr.startswith('__')]
attributes

['DEMA', 'EMA', 'KAMA', 'MAMA', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', '_lookup']

In [29]:
MA_Type.__dict__

{'_lookup': {0: 'Simple Moving Average',
  1: 'Exponential Moving Average',
  2: 'Weighted Moving Average',
  3: 'Double Exponential Moving Average',
  4: 'Triple Exponential Moving Average',
  5: 'Triangular Moving Average',
  6: 'Kaufman Adaptive Moving Average',
  7: 'MESA Adaptive Moving Average',
  8: 'Triple Generalized Double Exponential Moving Average'}}

In [30]:
print(MA_Type[MA_Type.DEMA])

Double Exponential Moving Average


Set timeperiod=20 and matype=MA_Type.EMA

In [31]:
#upper, middle, lower = BBANDS(ts, timeperiod=20, matype=MA_Type.EMA)
#(for some reason, the abstract API doesn't work for BBANDS, so use the function API)

upper, middle, lower = talib.BBANDS(ts.close, timeperiod=20, matype=MA_Type.EMA)
ts['upper'] = upper; ts['middle'] = middle; ts['lower'] = lower
ts.tail()

,high,low,open,close,volume,adj_close,sma200,cci,upper,middle,lower
date,,,,,,,,,,,
2022-11-17,394.95,390.14,390.46,394.24,74496300.00,394.24,406.09,57.02,402.70,385.47,368.25
2022-11-18,397.81,393.04,397.74,396.03,92783500.00,396.03,405.83,67.28,403.68,386.48,369.28
2022-11-21,395.82,392.66,394.64,394.59,51243200.00,394.59,405.57,50.60,404.55,387.25,369.96
2022-11-22,400.07,395.15,396.63,399.90,60429000.00,399.90,405.30,71.92,406.67,388.46,370.25
2022-11-23,402.93,399.31,399.55,402.42,68161400.00,402.42,405.07,93.21,408.95,389.79,370.63
